In [1]:
from waylay import WaylayClient, RestResponseError
waylay_client = WaylayClient.from_profile('demo')

# Waylay Python SDK - Making REST calls

The Waylay Python SDK exposes a number of _REST Services_, each a collection of _REST Resources_ that have _action methods_.

For an overview, see 
> `[Enterprise]` https://docs.waylay.io/#/api/sdk/python

In most cases these _action methods_ directly correspond to an underlying REST endpoint of the waylay platform.

In general, the Python SDK takes care of
* authentication
* the endpoint url to call, and HTTP method to use
* converting JSON responses to Python objects, extracting the relevant data
* converting Python request body objects to JSON
* handling errors 

Check out the documentation sites ([enterprise](https://docs.waylay.io/#/api/sdk/python)) on

* how requests and response are exactly represented as json
* which additional parameters can be provided
* what the preconditions and effects are of your call

### request arguments
These are the general rules for making requests, corresponding to the REST documentation of the underlying REST action:
* url **path parameters** are passed as **positional arguments**:
  ###### example

  > `waylay_client.queries.query.get('151CF-temperature')` 

  will use 151CF-temperature to construct a request to fetch this query definition

  > `GET .../config/query/151CF-temperature` 

* **query parameters** belong in a named **`params` argument**:
  ###### example
  > `waylay_client.queries.query.execute('151CF-temperature', params={'from':'2021-03-01T10:00:00+00:00'})`
  
  will bind the `from` parameter

* **request objects** are passed into a **`body` argument**:
  ###### example
  > `waylay_client.queries.query.execute(body={'resource':'RDJ_89839','metric':'revolutions'})`}|

### response handling
These are the general rules on how the SDK treats results of REST calls:
* Only successfull responses (HTTP status `2XX`) return a result, other responses raise an exception.
* The SDK extracts the most relevant part of the (json) response body. You can use the `select_path` argument to override this behaviour.
* When the REST call produces **timeseries data**, the SDK will return a **pandas DataFrame**.
* By providing the  `raw=True` argument, you instruct the SDK to skip all error and response handling, and **just return the HTTP response**.

### error handling

The SDK uses the following exception hierarchy to notify problems. These exception classes belong to the `waylay.exception` module. They all descend from a `WaylayError` base class.

| exception class | raised in case of |
| --------------- | ----------- |
| `AuthError` | Waylay authentication errors. |
| `ConfigError` |  Waylay client configuration errors. |
| `RequestError` | Errors in tools and utilities that are not directly related to a REST call. |
| `RestRequestError` | Failure to prepare a REST call. |
| `RestResponseError` | Wraps the result of a failed REST call. |
| `RestResponseParseError` | Failure to parse the result of a succesfull REST call. |

Errors of type `RestResponse` have the following attributes that you can use to handle problems:

* `response` contains the full HTTP Response object of the REST call, which lets you inspect the status code (`response.status_code`) , response body (`response.body`) and other attributes such as _headers_.
* `message` will give you the most relevant error message




### http response information

Most _action methods_ support a `raw=True` parameter. This will prevent exception handling and parsing of the REST call by the SDK. The unparsed result and http response information is returned in a _Response_ object with attributes
 * `body` : the result data (json and csv data is parsed to python data structures)
 * standard http information such as `url`, `method`, `headers`, `status_code`, `client_response`

In [2]:
waylay_client.queries.query.list()

['151CF-temperature',
 '151CF-temperature-demo',
 '151CF-temperature-wrong',
 ' battery-life-testset',
 'battery-life-testset',
 'battery-life-testset2',
 'battery-life-testset-sdk0.3',
 'BearingQuery',
 'BearingQuery_EngineA',
 'BearingQuery_EngineX']

In [3]:
http_response = waylay_client.queries.query.get('battery-life-testset', raw=True)
{
    'url':http_response.url,
    'method':http_response.method,
    'content-type':http_response.headers['content-type'],
    'status_code':http_response.status_code,
    'name':http_response.body['name'],
}


{'url': 'https://api.waylay.io/queries/v1/query/battery-life-testset',
 'method': 'GET',
 'content-type': 'application/json',
 'status_code': 200,
 'name': 'battery-life-testset'}

### access http response information from an error
When an request is unsuccessfull, the client will raise an exception. 

These exceptions (from the `waylay.exceptions` module) are either (instances of subclasses of)
* a `RestRequestError` that indicates a problem before sending an api call to waylay (e.g. when input argument conversion fails)
* a `RestResponseError` that reports a problem from or after the api call to waylay. This exception gives you access to the underlying response (`response` attribute)
  * a `RestResponseParseError` error indicates a problem in processing a succesfull response from the waylay platform. All other `RestResponseError` will come from errors reported by the waylay platform itself (http status code above the `200` range).
  
Other errors can occur (such as standard python `ValueError`,`TypeError` or `AttributeError`) but these will normally indicate a programming error. Networking failures will normally result in a `ClientConnectionError`

In [4]:
# try to get the representation of a `query` entity that does not exist. 
# this will result in a `404 NOT FOUND` error 
try:
   waylay_client.queries.query.get('where are you???')
except RestResponseError as exc:
   print(exc.message)
   print(exc.response.status_code)
   print(exc.response.url)
    


operation=not_found_error
404
https://api.waylay.io/queries/v1/query/where%20are%20you?%3F%3F=


## accessing the complete response
The SDK extracts the most relevant part of the (json) response body. You can use the `select_path` argument to override this behaviour

In [5]:
# the `query` entity in the REST response is given as default response on the SDK action
waylay_client.queries.query.get('battery-life-testset')

{'from': '2018-12-21T08:00:00Z',
 'data': [{'metric': 'cycle_number', 'resource': 'battery-life-testset'},
  {'metric': 'policy', 'resource': 'battery-life-testset'},
  {'metric': 'IR', 'resource': 'battery-life-testset'},
  {'metric': 'QD', 'resource': 'battery-life-testset'},
  {'metric': 'Discharge_time', 'resource': 'battery-life-testset'},
  {'metric': 'Qdlin_mean', 'resource': 'battery-life-testset'},
  {'metric': 'Qdlin_first', 'resource': 'battery-life-testset'},
  {'metric': 'Qdlin_last', 'resource': 'battery-life-testset'},
  {'metric': 'Qdlin_median', 'resource': 'battery-life-testset'},
  {'metric': 'Qdlin_25p', 'resource': 'battery-life-testset'},
  {'metric': 'Qdlin_75p', 'resource': 'battery-life-testset'},
  {'metric': 'Qdlin_0.1', 'resource': 'battery-life-testset'},
  {'metric': 'Qdlin_0.9', 'resource': 'battery-life-testset'},
  {'metric': 'Vdlin_mean', 'resource': 'battery-life-testset'},
  {'metric': 'Vdlin_first', 'resource': 'battery-life-testset'},
  {'metric': 

In [6]:
waylay_client.queries.query.get('battery-life-testset', select_path=False)

{'_links': {'self': {'href': 'https://api.waylay.io/queries/v1/query/battery-life-testset'},
  'execute': {'href': 'https://api.waylay.io/queries/v1/data/battery-life-testset'}},
 'attrs': {'created': '2021-11-08T12:39:27.625655+00:00',
  'created_by': 'users/f16ccae4-8f0f-4f39-912b-5c2cb13ff6b6',
  'modified': '2021-11-08T12:39:27.625655+00:00',
  'modified_by': 'users/f16ccae4-8f0f-4f39-912b-5c2cb13ff6b6',
  'path': '/tsa/query/battery-life-testset'},
 'name': 'battery-life-testset',
 'meta': {},
 'query': {'data': [{'metric': 'cycle_number',
    'resource': 'battery-life-testset'},
   {'metric': 'policy', 'resource': 'battery-life-testset'},
   {'metric': 'IR', 'resource': 'battery-life-testset'},
   {'metric': 'QD', 'resource': 'battery-life-testset'},
   {'metric': 'Discharge_time', 'resource': 'battery-life-testset'},
   {'metric': 'Qdlin_mean', 'resource': 'battery-life-testset'},
   {'metric': 'Qdlin_first', 'resource': 'battery-life-testset'},
   {'metric': 'Qdlin_last', 'reso

In [7]:
waylay_client.queries.query.get('battery-life-testset', select_path=['attrs','modified'])

'2021-11-08T12:39:27.625655+00:00'

## customise or replace dataframe conversions
The methods of the `queries.query` that return timeseries data, will normally return their data as a pandas `DataFrame`. These have immediate display support in jupyter notebooks:

In [8]:
waylay_client.queries.query.execute(
    'battery-life-testset', 
    params={ "until": "16-08-2021" }
)

resource                  battery-life-testset                             \
metric                            cycle_number policy        IR        QD   
timestamp                                                                   
2020-10-09 17:00:00+00:00                749.0    4.0  0.015722  1.034505   
2020-10-09 18:00:00+00:00               1177.0    3.0  0.015417  1.012836   
2020-10-09 19:00:00+00:00               1316.0    2.0  0.016785  1.028384   
2020-10-09 20:00:00+00:00                908.0    4.0  0.016336  0.995217   
2020-10-09 21:00:00+00:00                494.0    3.0  0.015234  1.048616   
...                                        ...    ...       ...       ...   
2020-12-31 20:00:00+00:00                254.0    3.0  0.017570  1.048546   
2020-12-31 21:00:00+00:00                418.0    4.0  0.019608  0.966915   
2020-12-31 22:00:00+00:00                995.0    4.0  0.018859  0.934390   
2020-12-31 23:00:00+00:00                462.0    5.0  0.019361  0.922506   
2021-01-01 00:00:00+00:00                906.0    3.0  0.016833  1.046472   

resource                                                                    \
metric                    Discharge_time Qdlin_mean Qdlin_first Qdlin_last   
timestamp                                                                    
2020-10-09 17:00:00+00:00      13.639208   0.692338    0.000090   1.001148   
2020-10-09 18:00:00+00:00      13.490242   0.687073    0.000143   0.990104   
2020-10-09 19:00:00+00:00      13.706227   0.702581    0.000153   1.004779   
2020-10-09 20:00:00+00:00      12.673982   0.628634    0.000274   0.930365   
2020-10-09 21:00:00+00:00      14.011083   0.723742    0.000020   1.027848   
...                                  ...        ...         ...        ...   
2020-12-31 20:00:00+00:00      13.406115   0.656861    0.000044   0.983731   
2020-12-31 21:00:00+00:00      11.582938   0.554695    0.000120   0.849428   
2020-12-31 22:00:00+00:00      11.714752   0.571552    0.000363   0.858524   
2020-12-31 23:00:00+00:00      11.113678   0.533706    0.000006   0.814875   
2021-01-01 00:00:00+00:00      13.908863   0.703591    0.000005   1.020206   

resource                                                              \
metric                    Qdlin_median Qdlin_25p Qdlin_75p Qdlin_0.1   
timestamp                                                              
2020-10-09 17:00:00+00:00     0.918587  0.363698  0.986413     219.0   
2020-10-09 18:00:00+00:00     0.914639  0.360545  0.970643     219.0   
2020-10-09 19:00:00+00:00     0.941263  0.388770  0.989392     219.0   
2020-10-09 20:00:00+00:00     0.835276  0.257268  0.915655     229.0   
2020-10-09 21:00:00+00:00     0.964289  0.418294  1.011859     213.0   
...                                ...       ...       ...       ...   
2020-12-31 20:00:00+00:00     0.876598  0.230785  0.969464     233.0   
2020-12-31 21:00:00+00:00     0.746234  0.123312  0.833388     245.0   
2020-12-31 22:00:00+00:00     0.770804  0.165150  0.838797     239.0   
2020-12-31 23:00:00+00:00     0.717896  0.120914  0.798694     246.0   
2021-01-01 00:00:00+00:00     0.941148  0.350603  0.998220     221.0   

resource                                                               \
metric                    Qdlin_0.9 Vdlin_mean Vdlin_first Vdlin_last   
timestamp                                                               
2020-10-09 17:00:00+00:00     461.0       2.75         3.5        2.0   
2020-10-09 18:00:00+00:00     468.0       2.75         3.5        2.0   
2020-10-09 19:00:00+00:00     421.0       2.75         3.5        2.0   
2020-10-09 20:00:00+00:00     704.0       2.75         3.5        2.0   
2020-10-09 21:00:00+00:00     388.0       2.75         3.5        2.0   
...                             ...        ...         ...        ...   
2020-12-31 20:00:00+00:00     547.0       2.75         3.5        2.0   
2020-12-31 21:00:00+00:00       0.0       2.75         3.5        2.0   
2020-12-31 22:00

You can use the `response_constructor` parameter to replace the dataframe constructor with your own method. 

Other _action methods_ that apply conversions to raw json data structures normally support this same optional parameter.

A falsy value will let the method return just the json data payload (as python object).

In [9]:
waylay_client.queries.query.execute(
    'battery-life-testset', 
    params={'window': 'PT1H'},
    response_constructor=False
)

[{'columns': ['timestamp',
   {'resource': 'battery-life-testset', 'metric': 'cycle_number'},
   {'resource': 'battery-life-testset', 'metric': 'policy'},
   {'resource': 'battery-life-testset', 'metric': 'IR'},
   {'resource': 'battery-life-testset', 'metric': 'QD'},
   {'resource': 'battery-life-testset', 'metric': 'Discharge_time'},
   {'resource': 'battery-life-testset', 'metric': 'Qdlin_mean'},
   {'resource': 'battery-life-testset', 'metric': 'Qdlin_first'},
   {'resource': 'battery-life-testset', 'metric': 'Qdlin_last'},
   {'resource': 'battery-life-testset', 'metric': 'Qdlin_median'},
   {'resource': 'battery-life-testset', 'metric': 'Qdlin_25p'},
   {'resource': 'battery-life-testset', 'metric': 'Qdlin_75p'},
   {'resource': 'battery-life-testset', 'metric': 'Qdlin_0.1'},
   {'resource': 'battery-life-testset', 'metric': 'Qdlin_0.9'},
   {'resource': 'battery-life-testset', 'metric': 'Vdlin_mean'},
   {'resource': 'battery-life-testset', 'metric': 'Vdlin_first'},
   {'resourc

In [10]:
# return a map with timestamps as keys, the observation values as value. 
waylay_client.queries.query.execute(
    'battery-life-testset', 
    params={ 'window': 'PT1H'},
    response_constructor=lambda d: { row[0] : row[1:] for row in d[0]['data'] }
)

{1545379200000: [438.0,
  3.0,
  0.019218624,
  0.93977582,
  11.503518333333588,
  0.5509009954718908,
  0.00036294473,
  0.84380955,
  0.7399182942083906,
  0.1194592173219805,
  0.829345337379427,
  246.0,
  0.0,
  2.75,
  3.5,
  2.0,
  35.36847819908963,
  30.341303,
  38.802608],
 1545382800000: [649.0,
  4.0,
  0.015047961,
  1.0383887,
  13.77363166666667,
  0.7047024719081196,
  8.3995874e-06,
  1.0104315,
  0.9334726571681689,
  0.3985925587172832,
  0.9958976790638401,
  215.0,
  434.0,
  2.75,
  3.5,
  2.0,
  37.36753921050169,
  34.22842984504116,
  39.25697056792589]}

In [11]:
import numpy as np
# return timeseries data as a numpy array, transposed as an array per series 
waylay_client.queries.query.execute(
   'battery-life-testset', 
    params={ 'window': 'PT1H'},
    response_constructor=lambda d: np.transpose(d[0]['data'])
)

array([[1.54537920e+12, 1.54538280e+12],
       [4.38000000e+02, 6.49000000e+02],
       [3.00000000e+00, 4.00000000e+00],
       [1.92186240e-02, 1.50479610e-02],
       [9.39775820e-01, 1.03838870e+00],
       [1.15035183e+01, 1.37736317e+01],
       [5.50900995e-01, 7.04702472e-01],
       [3.62944730e-04, 8.39958740e-06],
       [8.43809550e-01, 1.01043150e+00],
       [7.39918294e-01, 9.33472657e-01],
       [1.19459217e-01, 3.98592559e-01],
       [8.29345337e-01, 9.95897679e-01],
       [2.46000000e+02, 2.15000000e+02],
       [0.00000000e+00, 4.34000000e+02],
       [2.75000000e+00, 2.75000000e+00],
       [3.50000000e+00, 3.50000000e+00],
       [2.00000000e+00, 2.00000000e+00],
       [3.53684782e+01, 3.73675392e+01],
       [3.03413030e+01, 3.42284298e+01],
       [3.88026080e+01, 3.92569706e+01]])